# MultiQuery

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192"
)

In [14]:
from langchain.prompts import ChatPromptTemplate

# multi query: different perspective of one question
template = '''
You are an AI Language model. Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspective on the user question, your goal is to help the user overcome some of the limitations of the distance based similarity search. Provide these alternative questions separated by newlines. Just answer the questions nothing else. Original question: {question}'''

prompt_multi_query = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_multi_query
    | llm
    | (lambda x: x.content)
    | (lambda x: x.split("\n"))
)


In [15]:
generate_queries

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nYou are an AI Language model. Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspective on the user question, your goal is to help the user overcome some of the limitations of the distance based similarity search. Provide these alternative questions separated by newlines. Just answer the questions nothing else. Original question: {question}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FB14493A40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FB144B43B0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))
| RunnableLambda(...)
| 

In [16]:
generate_queries.invoke("How can I improve my learning?")

['How can I enhance my academic performance?',
 '',
 'What strategies can I use to boost my learning outcomes?',
 '',
 'What are some effective methods for improving my knowledge retention?',
 '',
 'What can I do to optimize my study habits and achieve better results?',
 '',
 'What are the most effective techniques for accelerating my learning process?']

In [22]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# embedding
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
hf_embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [26]:
len(hf_embeddings.embed_query("How can I improve my learning?"))

384

In [28]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./agent.txt")
document = loader.load()

In [35]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
document_split = splitter.split_documents(document)

In [36]:
document_split

[Document(metadata={'source': './agent.txt'}, page_content='What are AI Agents?\nAn artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals. For example, consider a contact center AI agent that wants to resolves customer queries. The agent will automatically ask the customer different questions, look up information in internal documents, and respond with a solution. Based on the customer responses, it determines if it can resolve the query itself or pass it on to a human.\n\nWhat are the key principles that define AI agents?\nAll software autonomously completes different tasks as determined by the software developer. So, what makes AI or intelligent agents special?'),
 Document(metadata={'source': './agent.txt'}, page_content="What are the 

In [38]:
from langchain.vectorstores import FAISS

#!pip install faiss-cpu

vectorstore = FAISS.from_documents(
    documents=document_split,
    embedding=hf_embeddings,
)

In [39]:
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FB49776DE0>, search_kwargs={})

In [47]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """get unique union of retrieved content"""
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]

question = "Ethical consideration in ai"
chain = generate_queries | retriever.map() | get_unique_union

In [48]:
docs = chain.invoke(question)
docs

[Document(metadata={'source': './agent.txt'}, page_content="What are the benefits of using AI agents?\nAI agents can improve your business operations and your customers' experiences.\n\nImproved productivity\nAI agents are autonomous intelligent systems performing specific tasks without human intervention. Organizations use AI agents to achieve specific goals and more efficient business outcomes. Business teams are more productive when they delegate repetitive tasks to AI agents. This way, they can divert their attention to mission-critical or creative activities, adding more value to their organization.\n\nReduced costs\nBusinesses can use intelligent agents to reduce unnecessary costs arising from process inefficiencies, human errors, and manual processes. You can confidently perform complex tasks because autonomous agents follow a consistent model that adapts to changing environments."),
 Document(metadata={'source': './agent.txt'}, page_content='Informed decision-making\nAdvanced i

In [52]:
from operator import itemgetter

data = [
    {"name": "Alice", "age": 10},
    {"name": "Bob", "age": 12},
    {"name": "Charlie", "age": 11},
]

names = list(map(itemgetter("name"), data))

print(names)

['Alice', 'Bob', 'Charlie']


In [49]:
from  operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the following question: {question}:< context given > Context: {context} <context> just answer the question nothing else"""

prompt = ChatPromptTemplate.from_template(template)
final_rag_chain = (
    {"context": chain, "question": itemgetter("question")}
    | prompt
    | llm
    | (lambda x: x.content)
)

final_rag_chain.invoke({"question": question})

'Ethical considerations in AI agents include:\n\n* Data privacy concerns\n* Biased or inaccurate results\n* Technical complexities\n* Limited compute resources'

# RAG Fusion
Reciprocal RAG Fusion

EXPLANATION:

**According to Question1**
Order of doc by relevance
1. Doc1
2. Doc2
3. Doc3
4. Doc4

**According to Q2**
1. Doc3
2. Doc1
3. Doc2
4. Doc4

**According to Q3**
1. Doc4
2. Doc1
3. Doc3
4. Doc2

Rank Positions

- Doc1:
 -- Question 1 rank: 1
 -- Question 2 rank: 2
 -- Question 3 rank: 2
- Doc2:
 -- Question 1 rank: 2
 -- Question 2 rank: 3
 -- Question 3 rank: 4
- Doc3:
 -- Question 1 rank: 3
 -- Question 2 rank: 1
 -- Question 3 rank: 3
- Doc4:
 -- Question 1 rank: 4
 -- Question 2 rank: 4
 -- Question 3 rank: 1

Reciprocal RAG Fusion

using k = 80 (arbitrary value)

RRF(Docn): sum[from i=1 to p where p = num of question](1 / (k + n))

##### Doc1
- Reciprocal Rank (Question1): 1 / (80 + 1) = 1 / 81
- Reciprocal Rank (Question2): 1 / (80 + 2) = 1 / 82
- Reciprocal Rank (Question3): 1 / (80 + 3) = 1 / 83
- RRF(Doc1): 1 / 81 + 1 / 82 + 1 / 83

Based on RRF score rank the document.

In [53]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=80):
    """Reciprocal rank fusion that takes multiple lists of ranked documents and parameter k to return a single list of ranked documents"""
    fused_scores = {}
    
    # iterate over each list of ranked document
    for docs in results:
        # iterate
        for rank, doc in enumerate(docs):
            # document to a string format to use as a key ( to json )
            doc_str = dumps(doc)
            # if the document is not in the fused scores, add it with score 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
                
            # retrieve the current score of the document
            previous_score = fused_scores[doc_str]
            # updat the score
            fused_scores[doc_str] = previous_score + 1 / (rank + k)
            
    # sort the fused scores by the score in desscending order to get the final ranked list
    reranked_results = [
        (loads(doc), score) for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    # return the final result
    return reranked_results
            

In [54]:
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

retrieval_chain_rag_fusion

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nYou are an AI Language model. Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspective on the user question, your goal is to help the user overcome some of the limitations of the distance based similarity search. Provide these alternative questions separated by newlines. Just answer the questions nothing else. Original question: {question}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FB14493A40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FB144B43B0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))
| RunnableLambda(...)
| 

In [56]:
docs = retrieval_chain_rag_fusion.invoke({"question": question})
docs

[(Document(metadata={'source': './agent.txt'}, page_content='Ethical challenges\nIn certain circumstances, deep learning models may produce unfair, biased, or inaccurate results. Applying safeguards, such as human reviews, ensures customers receive helpful and fair responses from the agents deployed. \n\nTechnical complexities \nImplementing advanced AI agents requires specialized experience and knowledge of machine learning technologies. Developers must be able to integrate machine learning libraries with software applications and train the agent with enterprise-specific data. \n\nLimited compute resources\nTraining and deploying deep learning AI agents requires substantial computing resources. When organizations implement these agents on-premise, they must invest in and maintain costly infrastructure that is not easily scalable.'),
  0.13749999999999998),
 (Document(metadata={'source': './agent.txt'}, page_content='Implement tasks\nWith sufficient data, the AI agent methodically impl

In [57]:
from langchain_core.runnables import RunnablePassthrough

template = '''answer the following question: {question} given context: {context} only answer the question nothing else.'''

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | (lambda x: x.content)
)

final_rag_chain.invoke({"question": question})

'Ethical considerations in AI given the context are:\n\n* Unfair, biased, or inaccurate results from deep learning models\n* Data privacy concerns when acquiring, storing, and moving massive volumes of data\n* Limited compute resources required for training and deploying deep learning AI agents'

# Decomposition

In [105]:
from langchain.prompts import ChatPromptTemplate

template = """
You are a assistant that generates multiple sub-questions related to an input question. The goal is to break down the input into a set of sub-problems that can be solved independently. Generate three search queries related to: {question}. Response: just response the queries only """

prompt_decomposition = ChatPromptTemplate.from_template(template)

In [106]:
generate_queries_decomposition = (
    prompt_decomposition
    | llm
    | (lambda x: x.content)
    | (lambda x: x.split("\n"))
)
generate_queries_decomposition

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nYou are a assistant that generates multiple sub-questions related to an input question. The goal is to break down the input into a set of sub-problems that can be solved independently. Generate three search queries related to: {question}. Response: just response the queries only '), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FB14493A40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FB144B43B0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))
| RunnableLambda(...)
| RunnableLambda(...)

In [102]:
question = "What is llm agent in ai"

In [108]:
generate_queries_decomposition.invoke(question)

['Here are three sub-questions related to "What is LLM Agent in AI":',
 '',
 '1. What is Large Language Model (LLM) in AI?',
 '2. What is the role of an Agent in AI systems?',
 '3. How does a Language Model Agent (LLM Agent) integrate these two concepts in AI applications?']

In [110]:
generate_queries_decomposition.invoke("How can I improve my learning?")

['Here are three sub-questions related to "How can I improve my learning?":',
 '',
 '1. What strategies can I use to enhance my focus and concentration during studying?',
 '2. How can I effectively organize and manage my time to optimize my learning?',
 '3. What techniques can I use to retain information and recall it more easily?']

In [115]:
template = """
Here is the question you need to answer
\n---\n {question}\n---\n
Here is available background question and answer pairs:
\n---\n {qa}\n---\n
Here is other context relevant to the question:
\n---\n {context}\n---\n
Use the above context and any background question and answer pair to answer the question {question}"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [117]:
decomposition_prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'qa', 'question'], input_types={}, partial_variables={}, template='\nHere is the question you need to answer\n\n---\n {question}\n---\n\nHere is available background question and answer pairs:\n\n---\n {qa}\n---\n\nHere is other context relevant to the question:\n\n---\n {context}\n---\n\nUse the above context and any background question and answer pair to answer the question {question}'), additional_kwargs={})]

In [ ]:
from operator import itemgetter

In [122]:
def format_qa_pairs(questions, answers):
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question{i}: {question}\nAnswer {i}: {answer}\n"
    # return the string
    return formatted_string.strip()


In [123]:
from langchain_core.output_parsers import StrOutputParser

In [127]:
q_a_pairs = ""
question = "What is an ai agent?"

questions = generate_queries_decomposition.invoke(question)

for q in questions:
    c = (
        {"context": itemgetter("question") | retriever,
         "question": itemgetter("question"),
         "qa": itemgetter("qa")}
        | decomposition_prompt
        | llm
        | StrOutputParser()
    )
    answer = c.invoke({"question": q, "qa": q_a_pairs})
    q_a_pair = format_qa_pairs(q, answer)
    q_a_pairs += "\n\n" + q_a_pair

In [128]:
answer

'Based on the provided context and background question and answer pairs, I will answer the question 3.\n\nThe context discusses AI agents as software programs that can interact with their environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. AI agents are rational agents that make rational decisions based on their perceptions and data to produce optimal performance and results.\n\nFrom the context, I can identify the following types of AI agents:\n\n1. **Stationary Agents**: These agents are software programs that operate within a specific environment or system, collecting data and performing tasks without moving physically. Examples include chatbots, robotic agents that collect sensor data, and software agents that analyze internal documents.\n2. **Mobile Agents**: These agents are software programs that can move between different environments or systems, collecting data and performing tasks on the move. Examples include self-driving

# answer individually

In [129]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [130]:
promt_rag = hub.pull("rlm/rag-prompt")

In [132]:
def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    """Retrieve and RAG"""
    sub_questions = sub_question_generator_chain.invoke(question)
    rag_results = []
    for q in sub_questions:
        retreived_doc = retriever.get_relevant_documents(q)
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"question": q, "context": retreived_doc})
        
        rag_results.append(answer)
    return rag_results, sub_questions

In [133]:
answers, questions = retrieve_and_rag(question, promt_rag, generate_queries_decomposition)

C:\Users\acer\AppData\Local\Temp\ipykernel_13736\2762158091.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retreived_doc = retriever.get_relevant_documents(q)


In [134]:
def format_qa_pairs(questions, answers):
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\n\n Answer: {answer}\n\n"
    return formatted_string.strip()

In [135]:
context = format_qa_pairs(questions, answers)

In [136]:
context

'Question 1: Here are three sub-questions related to "What is an AI Agent?":\n\n Answer: What makes AI or intelligent agents special is that they are rational agents that make rational decisions based on their perceptions and data to produce optimal performance and results.\n\nQuestion 2: \n\n Answer: The key components of AI agent architecture are not explicitly mentioned in the provided context.\n\nQuestion 3: 1. What is the definition of an artificial intelligence (AI) agent?\n\n Answer: An artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. The agent makes rational decisions based on its perceptions and data to produce optimal performance and results.\n\nQuestion 4: 2. What are the characteristics of a typical AI agent?\n\n Answer: A typical AI agent is a software program that can interact with its environment, collect data, and use the data to p

In [138]:
template = """
Here is a set of qa pairs: {context}
Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

rag = (
    prompt | llm | StrOutputParser()
)

In [139]:
rag

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nHere is a set of qa pairs: {context}\nUse these to synthesize an answer to the question: {question}\n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FB14493A40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FB144B43B0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [140]:
rag.invoke({"question": question, "context": context})

'Based on the provided QA pairs, here is a synthesized answer to the question "What is an AI Agent?":\n\nAn AI agent is a software program that can interact with its environment, collect data, and use that data to perform self-determined tasks to meet predetermined goals. It is a rational agent that makes decisions based on its perceptions and data to produce optimal performance and results. This means that AI agents can sense their environment through physical or software interfaces, apply the data they collect to make informed decisions, and adjust their actions accordingly. Unlike other types of artificial intelligence systems, such as expert systems or neural networks, AI agents have the ability to interact with their environment and make decisions based on their perceptions and data. Overall, AI agents are software programs that can act autonomously and make rational decisions to achieve specific goals.'